In [ ]:
import pandas as pd
df = pd.read_parquet('metadata_files/metadata.parquet', engine='pyarrow')

In [ ]:
df

In [ ]:
df.shape

In [ ]:
len(df["prompt"].unique())

In [ ]:
# read 1000 firstnames
names = []
f = open("1000_firstnames.txt", "r")
for x in f.readlines():
  n = x[0:-1]
  if len(n) >= 3:
      names.append(n)
# print(names)

import time

counter = 0
starttime = time.time()

import re
def name(str):
    global counter
    if counter > 1000:
        return "False"
    if counter % 1000== 0:
        print(f'{counter} {(time.time()-starttime)/60} minutes')
    counter += 1
    splits = [x for x in re.split(';|,|\.| |:', str) if len(x)>0]
    #print(splits)
    for n in names:
        for s in splits:
            if n == s.strip():
                #print(n)
                return n
    return "False"

df['firstname'] = df['prompt'].map(
        lambda p: name(p))

no_firstnames = df.loc[df['firstname'] == "False"]
firstnames = df.loc[df['firstname'] != "False"]

print(firstnames[["prompt", "firstname"]])

firstnames.to_parquet("firstnames.parquet")
firstnames.to_csv("firstnames.csv")


In [ ]:
df['artist'] = df['prompt'].apply(
        lambda query: 'greg' if "greg rutkowski" in query else query)

In [ ]:
given_authors = ["greg rutkowsky", "Vincent van Gogh", "Banksy", "Salvador Dalí"]
given_authors = [x.lower() for x in given_authors]

similarity_threshold = 0.6

from difflib import SequenceMatcher as SM

def author_included(ga, parts):
    author_components = ga.split(" ")
    components_found = 0
    for ac in author_components:
        for p in parts:
            # print(f'{p} {ac} {SM(None, p, ac).ratio()}')
            if SM(None, p, ac).ratio() >= similarity_threshold: ## fuzzy comparison possible here
                components_found += 1
                break
    if components_found == len(author_components):
        return True
    else:
        return False

import re
def extract_authors(str):
    authors = []
    parts = re.split(';|,|\.| |:', str.lower())
    print(parts)
    for ga in given_authors:
        if author_included(ga, parts):
            authors.append(ga)
    
    return authors

print(extract_authors("artwork by greg rutkowsky"))
print()
print(extract_authors("artwork, by greg rutzky"))
print()
print(extract_authors("artwork by greg rutkowsky and Salvador Dalí"))
print()
print(extract_authors("artwork by greg rutkowsky and Salvador Dalí and Banksy"))
print()
print(extract_authors("banksy"))


    

In [ ]:
gregs = df.loc[df['artist'] == "greg"]

In [ ]:
pd.options.display.max_colwidth = 1000

In [ ]:
print(gregs["prompt"])

In [ ]:
gregs.shape

In [ ]:
df['artist'] = df['prompt'].apply(
        lambda query: 'banksy' if "banksy" in query else query)
banksy = df.loc[df['artist'] == "banksy"]
banksy[["prompt", "artist"]]

In [ ]:
firstnames_reload = pd.read_parquet('firstnames.parquet', engine='pyarrow')
firstnames_reload # works as intended